<a href="https://colab.research.google.com/github/Mehshida/Mahshida/blob/main/Copy_of_MyoChallenge_ProstheticLeg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2024 NeurIPS - MyoChallenge

## <center> Welcome to the [**2024 NeurIPS - MyoChallenge:  Physiological Dexterity and Agility in Enhanced Humans**](https://sites.google.com/view/myosuite/myochallenge/myochallenge-2024) </center>

# 1. Setting the environment

In [3]:
!pip install myosuite==2.5.0
!pip install stable-baselines3[extra]  --quiet
!pip install tqdm  --quiet
!pip install sk-video
%env MUJOCO_GL=egl
import mujoco

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ...

### Define a method to show the rendering inside the Colab

In [4]:
from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width = 400):

  video_file = open(video_path, "r+b").read()

  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video autoplay width={video_width} controls><source src="{video_url}"></video>""")


### All the `MyoSuite` imports needed to run this tutorial

In [2]:
import myosuite
from myosuite.utils import gym
import skvideo.io
import numpy as np
import os
from stable_baselines3 import PPO
from tqdm import tqdm_notebook as tqdm

/usr/local/lib/python3.10/dist-packages/glfw/__init__.py:914: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)


MyoSuite:> Registering Myo Envs


## Creating the MyoChallenge '24 Environment and train your model

### Loading the Locomotion Challenge Env

As a basic example, we use PPO to train the locomotion environment on a task to walk over different types of terrain. Some helper functions are also provided for the environment initialization, so feel free to explore them.

In [1]:
# Create the MyoChallenge environment and train the model with PPO

"""
Preset environment modes are as given as ['init', 'random', 'osl_init']:
These can be activated by passing an argument "reset_type"

1. init - Resets the model in a neutral standing pose. OSL State Machine is initialized with 'e_stance'

2. random - Resets the model in a random pose. OSL State Machine initialized with 'e_stance'.
IMPT: State Machine not guranteed to be stable, since there joint positions do not match the thresholds in the state transitions.

3. osl_init - Resets the model with a pose sampled from a sample gait trajectory.
IMPT: State Machine initialized according to the pose sampled. Sampled poses are within state transition thresholds. More stable than "random"
"""

env = gym.make('myoChallengeRunTrackP1-v0', reset_type='random')
env.reset()

# Repeat 1000 time steps
for _ in range(1000):

    # Activate mujoco rendering window
    env.mj_render()


    # Get observation from the envrionment, details are described in the above docs
    obs = env.get_obs()
    current_time = obs['time']
    #print(current_time)


    # Take random actions
    action = env.action_space.sample()


    # Environment provides feedback on action
    next_obs, reward, terminated, truncated, info = env.step(action)


    # Reset training if env is terminated
    if terminated:
        next_obs, info = env.reset()

model = PPO("MlpPolicy", env, verbose=0)
model.learn(total_timesteps=100)

NameError: name 'gym' is not defined

In [2]:
# evaluate policy
all_rewards = []
for _ in tqdm(range(5)): # Randomization over different terrain types
  ep_rewards = []
  done = False
  obs = env.reset()
  while not done:
      obs = env.obsdict2obsvec(env.obs_dict, env.obs_keys)[1]
      # get the next action from the policy
      action, _ = model.predict(obs, deterministic=True)
      # take an action based on the current observation
      obs, reward, done, info, _ = env.step(action)
      ep_rewards.append(reward)
  all_rewards.append(np.sum(ep_rewards))
print(f"Average reward: {np.mean(all_rewards)} over 20 episodes")

NameError: name 'tqdm' is not defined

## Rendering your policy

You can render your policy on the task with the built-in renderer below.

In [ ]:
# Render trained policy
frames = []
for _ in tqdm(range(5)): # Randomization over different terrain types
  env.reset()
  ep_rewards = []
  done = False
  obs = env.reset()
  for _ in range(20):
      obs = env.obsdict2obsvec(env.obs_dict, env.obs_keys)[1]
      # get the next action from the policy
      action, _ = model.predict(obs, deterministic=True)
      geom_1_indices = np.where(env.sim.model.geom_group == 1)
      env.sim.model.geom_rgba[geom_1_indices, 3] = 0
      frame = env.sim.renderer.render_offscreen(
                        width=400,
                        height=400,
                        camera_id=1)
      frames.append(frame)
      # take an action based on the current observation
      obs, reward, done, info, _ = env.step(action)

env.close()

os.makedirs('videos', exist_ok=True)
# make a local copy
skvideo.io.vwrite('videos/test_policy.mp4', np.asarray(frames),outputdict={"-pix_fmt": "yuv420p"})
show_video('videos/test_policy.mp4')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-6-71d67c7be798>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _ in tqdm(range(5)): # Randomization over different terrain types


  0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.sim to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.sim` for environment variables or `env.get_wrapper_attr('sim')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/skvideo/io/ffmpeg.py:466: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  self._proc.stdin.write(vid.tostring())
/usr/local/lib/python3.10/dist-packages/skvideo/io/ffmpeg.py:466: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  self._proc.stdin.write(vid.tostring())
/usr/local/lib/python3.10/dist-packages/skvideo/io/ffmpeg.py:466: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  self._proc.stdin.write(vid.tostring())
/usr/local/lib/python3.10/dist-packages/skvideo/io/ffmpeg.py:466: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
